In [172]:
import pandas as pd
import requests
import pickle
from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
from langchain_openai import OpenAI

df = pd.read_csv("/Users/moksh/Desktop/NFLPlaybyPlay2015.csv")
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", api_key=key),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
)
agent.run("How many touchdowns did T.Brady throw after the bye week?")
headers = {"Ocp-Apim-Subscription-Key": ""}
data = {}
for year in range(2002,2024):
    year_format = str(year) + "REG"
    response =requests.get(f'https://api.sportsdata.io/v3/nfl/stats/json/PlayerGameStatsByWeek/{year_format}/1', headers=headers)
    data[year] = response.json()
    print(year)
    
player_ids = {}
for dict_ in data.values():
    df = pd.DataFrame(dict_)
    for i,row in df.iterrows():
        player_ids[row["Name"]] = row["PlayerID"]
        
player_ids_df = pd.DataFrame(columns=['name','id'])
player_ids_df['id'] = player_ids.values()
player_ids_df['first_name'] = [name.split()[0] for name in player_ids.keys()]
player_ids_df['last_name'] = [name.split()[1] for name in player_ids.keys()]
player_ids_df.drop(['name'], inplace=True, axis=1)
agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613", api_key=key),
    player_ids_df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    
)
agent.run("What is Johnson player id. Return just the id. If there are spelling mistakes, correct in your best judgement")


